In [ ]:
# Load the relevant packages.
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import anndata
from scipy import sparse

sc.settings.verbosity = 3 # Possible values: (0) errors, (1) warnings, (2) info, (3) hints
sc.settings.set_figure_params(dpi = 100, facecolor='white', fontsize=18, transparent=True)

from sccoda.util import comp_ana as mod
from sccoda.util import cell_composition_data as dat
from sccoda.util import data_visualization as viz

In [ ]:
# Load the processed fibroblast and immune data
results_directory = '../data/' # Initialise the results file

In [ ]:
# Load the data
results_name = 'integratedskindata.h5ad'
skin_merged = sc.read_h5ad(results_directory + results_name)
unwounded_merged = skin_merged[skin_merged.obs['sample'].isin(['UW P49', 'UW P21'])]

In [ ]:
# Do some renaming
unwounded_merged.obs['sub_sample'].cat.rename_categories({'UW_1_H20':'UW_P49_1',
                                                          'UW_1_P20':'UW_P21_1',
                                                          'UW_2_H20':'UW_P49_2',
                                                          'UW_2_P20':'UW_P21_2',
                                                          'UW_3_P20':'UW_P21_3'}, inplace=True)

unwounded_merged.obs['sub_sample'].cat.reorder_categories(['UW_P21_1', 'UW_P21_2',
                                                           'UW_P21_3', 'UW_P49_1',
                                                           'UW_P49_2'], inplace=True)



unwounded_merged.obs['leiden'].cat.rename_categories({'Epidermal 1':'EPI-1',
                                                      'Epidermal 2':'EPI-2',
                                                      'Epidermal 3':'EPI-3',
                                                      'Epidermal 4':'EPI-4',
                                                      'Epidermal 5':'EPI-5',
                                                      'Fibroblast 1':'FIB-1',
                                                      'Fibroblast 2':'FIB-2',
                                                      'Fibroblast 3':'FIB-3',
                                                      'Fibroblast 4':'FIB-4',
                                                      'Immune 1':'IMM-1',
                                                      'Immune 2':'IMM-2',
                                                      'Immune 3':'IMM-3',
                                                      'Pericyte':'PERI',
                                                      'Endothelial':'ENDO',
                                                      'Schwann':'SCH',
                                                      'Lymphatic endothelial':'LYM-E',
                                                      'Melanocyte':'MEL',
                                                      'Langerhans cell':'LC',
                                                      'Skeletal muscle':'SKEL',
                                                      'Smooth muscle':'SMC'}, inplace=True)

unwounded_merged.obs['leiden'].cat.reorder_categories(['EPI-1',
                                                       'EPI-2',
                                                       'EPI-3',
                                                       'EPI-4',
                                                       'EPI-5',
                                                       'FIB-1',
                                                       'FIB-2',
                                                       'FIB-3',
                                                       'FIB-4',
                                                       'IMM-1',
                                                       'IMM-2',
                                                       'IMM-3',
                                                       'PERI',
                                                       'ENDO',
                                                       'SCH',
                                                       'LYM-E',
                                                       'MEL',
                                                       'LC',
                                                       'SKEL',
                                                       'SMC'], inplace=True)

# Compare across all cell types

In [ ]:
unwounded_sccoda = dat.from_scanpy(unwounded_merged,
                                     cell_type_identifier='leiden',
                                     sample_identifier='sub_sample')
unwounded_sccoda.obs['Condition'] = pd.Series(['P21', 'P21', 'P21', 'P49', 'P49']).values

In [ ]:
model_uw = mod.CompositionalAnalysis(unwounded_sccoda, formula="Condition", reference_cell_type="automatic")
uw_sccoda_results = model_uw.sample_hmc()

In [ ]:
uw_sccoda_results.set_fdr(est_fdr=0.05)
uw_sccoda_results.credible_effects()

In [ ]:
uw_sccoda_results.set_fdr(est_fdr=0.01)
uw_sccoda_results.credible_effects()

In [ ]:
uw_sccoda_results.set_fdr(est_fdr=0.001)
uw_sccoda_results.credible_effects()

In [ ]:
# Plot the different proportions
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.family'] = 'Arial'
# fig, ax  = plt.subplots
ax = viz.boxplots(unwounded_sccoda, feature_name="Condition", y_scale='relative', add_dots=True, cmap='colorblind')
plt.ylim(-0.01, 0.65)
plt.xlabel('Cell cluster')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
for line in ax.get_lines():
    line.set_color('black')
    

# Epidermal cells only

In [ ]:
epidermal_merged = unwounded_merged[unwounded_merged.obs['leiden'].str.startswith('EPI-')]

epidermal_merged.obs['sub_sample'].cat.rename_categories({'UW_1_H20':'UW_P49_1',
                                                          'UW_1_P20':'UW_P21_1',
                                                          'UW_2_H20':'UW_P49_2',
                                                          'UW_2_P20':'UW_P21_2',
                                                          'UW_3_P20':'UW_P21_3'}, inplace=True)

epidermal_merged.obs['sub_sample'].cat.reorder_categories(['UW_P21_1', 'UW_P21_2',
                                                           'UW_P21_3', 'UW_P49_1',
                                                           'UW_P49_2'], inplace=True)

In [ ]:
# Define scCODA object
epidermal_sccoda = dat.from_scanpy(epidermal_merged,
                                     cell_type_identifier='leiden_sub',
                                     sample_identifier='sub_sample')
epidermal_sccoda.obs['Condition'] = pd.Series(['P21', 'P21', 'P21', 'P49', 'P49']).values

In [ ]:
# Train model
model_epi = mod.CompositionalAnalysis(epidermal_sccoda, formula="Condition", reference_cell_type="automatic")
epi_sccoda_results = model_epi.sample_hmc()

In [ ]:
epi_sccoda_results.set_fdr(est_fdr=0.05)
epi_sccoda_results.credible_effects()

In [ ]:
epi_sccoda_results.set_fdr(est_fdr=0.01)
epi_sccoda_results.credible_effects()

In [ ]:
epi_sccoda_results.set_fdr(est_fdr=0.001)
epi_sccoda_results.credible_effects()

In [ ]:
# Plot the different proportions
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.family'] = 'Arial'
# fig, ax  = plt.subplots
ax = viz.boxplots(epidermal_sccoda, feature_name="Condition", y_scale='relative', add_dots=True, cmap='colorblind')
plt.ylim(-0.01, 0.65)
plt.xlabel('Cell cluster')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
for line in ax.get_lines():
    line.set_color('black')
    

# Fibroblasts

In [ ]:
# Isolate fibroblasts this way to avoid the initial batch of RBCs
fibroblast_merged = unwounded_merged[unwounded_merged.obs['leiden'].str.startswith('FIB')]
fibroblast_merged = fibroblast_merged[fibroblast_merged.obs['leiden_sub'] != 'RBC']

fibroblast_merged.obs['sub_sample'].cat.rename_categories({'UW_1_H20':'UW_P49_1',
                                                          'UW_1_P20':'UW_P21_1',
                                                          'UW_2_H20':'UW_P49_2',
                                                          'UW_2_P20':'UW_P21_2',
                                                          'UW_3_P20':'UW_P21_3'}, inplace=True)

fibroblast_merged.obs['sub_sample'].cat.reorder_categories(['UW_P21_1', 'UW_P21_2',
                                                           'UW_P21_3', 'UW_P49_1',
                                                           'UW_P49_2'], inplace=True)

fibroblast_merged.obs['leiden_sub'].cat.reorder_categories(['DP', 'FIB-II', 'FIB-III', 'FIB-IV',
                                                            'FIB-VI', 'FIB-VII', 'FIB-VIII', 'FIB-IX',
                                                            'FIB-X'], inplace=True)

In [ ]:
fibroblast_sccoda = dat.from_scanpy(fibroblast_merged,
                                     cell_type_identifier='leiden_sub',
                                     sample_identifier='sub_sample')
fibroblast_sccoda.obs['Condition'] = pd.Series(['P21', 'P21', 'P21', 'P49', 'P49']).values

In [ ]:
# Train model
model_fib = mod.CompositionalAnalysis(fibroblast_sccoda, formula="Condition", reference_cell_type="automatic")
fib_sccoda_results = model_fib.sample_hmc()

In [ ]:
fib_sccoda_results.set_fdr(est_fdr=0.05)
fib_sccoda_results.credible_effects()

In [ ]:
fib_sccoda_results.set_fdr(est_fdr=0.01)
fib_sccoda_results.credible_effects()

In [ ]:
fib_sccoda_results.set_fdr(est_fdr=0.001)
fib_sccoda_results.credible_effects()

In [ ]:
# Plot the different proportions
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.family'] = 'Arial'
# fig, ax  = plt.subplots
ax = viz.boxplots(fibroblast_sccoda, feature_name="Condition", y_scale='relative', add_dots=True, cmap='colorblind')
plt.ylim(-0.01, 0.65)
plt.xlabel('Cell cluster')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
for line in ax.get_lines():
    line.set_color('black')
    

# Immune cells

In [ ]:
immune_merged = unwounded_merged[unwounded_merged.obs['leiden'].str.startswith('Immune')]
immune_merged = immune_merged[immune_merged.obs['leiden_sub'] != 'RBC']

immune_merged.obs['sub_sample'].cat.rename_categories({'UW_1_H20':'UW_P49_1',
                                                          'UW_1_P20':'UW_P21_1',
                                                          'UW_2_H20':'UW_P49_2',
                                                          'UW_2_P20':'UW_P21_2',
                                                          'UW_3_P20':'UW_P21_3'}, inplace=True)

immune_merged.obs['sub_sample'].cat.reorder_categories(['UW_P21_1', 'UW_P21_2',
                                                           'UW_P21_3', 'UW_P49_1',
                                                           'UW_P49_2'], inplace=True)

In [ ]:
immune_sccoda = dat.from_scanpy(immune_merged,
                                     cell_type_identifier='leiden_sub',
                                     sample_identifier='sub_sample')
immune_sccoda.obs['Condition'] = pd.Series(['P21', 'P21', 'P21', 'P49', 'P49']).values

In [ ]:
model_imm = mod.CompositionalAnalysis(immune_sccoda, formula="Condition", reference_cell_type="automatic")
immune_sccoda_results = model_imm.sample_hmc()

In [ ]:
immune_sccoda_results.set_fdr(est_fdr=0.05)
immune_sccoda_results.credible_effects()

In [ ]:
immune_sccoda_results.set_fdr(est_fdr=0.01)
immune_sccoda_results.credible_effects()

In [ ]:
immune_sccoda_results.set_fdr(est_fdr=0.001)
immune_sccoda_results.credible_effects()

In [ ]:
# Plot the different proportions
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.family'] = 'Arial'
# fig, ax  = plt.subplots
ax = viz.boxplots(immune_sccoda, feature_name="Condition", y_scale='relative', add_dots=True, cmap='colorblind')
plt.ylim(-0.01, 0.65)
plt.xlabel('Cell cluster')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
for line in ax.get_lines():
    line.set_color('black')
    

# Pericytes

In [ ]:
pericyte_merged = unwounded_merged[unwounded_merged.obs['leiden'] == 'PERI']

pericyte_merged.obs['sub_sample'].cat.rename_categories({'UW_1_H20':'UW_P49_1',
                                                          'UW_1_P20':'UW_P21_1',
                                                          'UW_2_H20':'UW_P49_2',
                                                          'UW_2_P20':'UW_P21_2',
                                                          'UW_3_P20':'UW_P21_3'}, inplace=True)

pericyte_merged.obs['sub_sample'].cat.reorder_categories(['UW_P21_1', 'UW_P21_2',
                                                           'UW_P21_3', 'UW_P49_1',
                                                           'UW_P49_2'], inplace=True)

In [ ]:
pericyte_sccoda = dat.from_scanpy(pericyte_merged,
                                     cell_type_identifier='leiden_sub',
                                     sample_identifier='sub_sample')
pericyte_sccoda.obs['Condition'] = pd.Series(['P21', 'P21', 'P21', 'P49', 'P49']).values

In [ ]:
model_peri = mod.CompositionalAnalysis(pericyte_sccoda, formula="Condition", reference_cell_type="automatic")
pericyte_sccoda_results = model_peri.sample_hmc()

In [ ]:
pericyte_sccoda_results.set_fdr(est_fdr=0.05)
pericyte_sccoda_results.credible_effects()

In [ ]:
pericyte_sccoda_results.set_fdr(est_fdr=0.01)
pericyte_sccoda_results.credible_effects()

In [ ]:
pericyte_sccoda_results.set_fdr(est_fdr=0.001)
pericyte_sccoda_results.credible_effects()

In [ ]:
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.family'] = 'Arial'
# fig, ax  = plt.subplots
ax = viz.boxplots(pericyte_sccoda, feature_name="Condition", y_scale='relative', add_dots=True, cmap='colorblind')
plt.ylim(-0.01, 0.9)
plt.xlabel('Cell cluster')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

for line in ax.get_lines():
    line.set_color('black')
# plt.show()

# Endothelial cells

In [ ]:
endothelial_merged = unwounded_merged[unwounded_merged.obs['leiden'].str.startswith('ENDO')]

endothelial_merged.obs['sub_sample'].cat.rename_categories({'UW_1_H20':'UW_P49_1',
                                                          'UW_1_P20':'UW_P21_1',
                                                          'UW_2_H20':'UW_P49_2',
                                                          'UW_2_P20':'UW_P21_2',
                                                          'UW_3_P20':'UW_P21_3'}, inplace=True)

endothelial_merged.obs['sub_sample'].cat.reorder_categories(['UW_P21_1', 'UW_P21_2',
                                                           'UW_P21_3', 'UW_P49_1',
                                                           'UW_P49_2'], inplace=True)

In [ ]:
endothelial_sccoda = dat.from_scanpy(endothelial_merged,
                                     cell_type_identifier='leiden_sub',
                                     sample_identifier='sub_sample')
endothelial_sccoda.obs['Condition'] = pd.Series(['P21', 'P21', 'P21', 'P49', 'P49']).values

In [ ]:
model_endo = mod.CompositionalAnalysis(endothelial_sccoda, formula="Condition", reference_cell_type="automatic")
endo_sccoda_results = model_endo.sample_hmc()

In [ ]:
endo_sccoda_results.set_fdr(est_fdr=0.05)
endo_sccoda_results.credible_effects()

In [ ]:
endo_sccoda_results.set_fdr(est_fdr=0.01)
endo_sccoda_results.credible_effects()

In [ ]:
endo_sccoda_results.set_fdr(est_fdr=0.001)
endo_sccoda_results.credible_effects()

In [ ]:
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.family'] = 'Arial'
# fig, ax  = plt.subplots
ax = viz.boxplots(endothelial_sccoda, feature_name="Condition", y_scale='relative', add_dots=True, cmap='colorblind')
plt.ylim(-0.01, 0.9)
plt.xlabel('Cell cluster')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

for line in ax.get_lines():
    line.set_color('black')
# plt.show()

# Schwann cells

In [ ]:
schwann_merged = unwounded_merged[unwounded_merged.obs['leiden'].str.startswith('SCH')]

schwann_merged.obs['sub_sample'].cat.rename_categories({'UW_1_H20':'UW_P49_1',
                                                          'UW_1_P20':'UW_P21_1',
                                                          'UW_2_H20':'UW_P49_2',
                                                          'UW_2_P20':'UW_P21_2',
                                                          'UW_3_P20':'UW_P21_3'}, inplace=True)

schwann_merged.obs['sub_sample'].cat.reorder_categories(['UW_P21_1', 'UW_P21_2',
                                                           'UW_P21_3', 'UW_P49_1',
                                                           'UW_P49_2'], inplace=True)

In [ ]:
schw_sccoda = dat.from_scanpy(schwann_merged,
                                     cell_type_identifier='leiden_sub',
                                     sample_identifier='sub_sample')
schw_sccoda.obs['Condition'] = pd.Series(['P21', 'P21', 'P21', 'P49', 'P49']).values

In [ ]:
model_schw = mod.CompositionalAnalysis(schw_sccoda, formula="Condition", reference_cell_type="automatic")
schw_sccoda_results = model_schw.sample_hmc()

In [ ]:
schw_sccoda_results.set_fdr(est_fdr=0.05)
schw_sccoda_results.credible_effects()

In [ ]:
schw_sccoda_results.set_fdr(est_fdr=0.01)
schw_sccoda_results.credible_effects()

In [ ]:
schw_sccoda_results.set_fdr(est_fdr=0.001)
schw_sccoda_results.credible_effects()

In [ ]:
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.family'] = 'Arial'
# fig, ax  = plt.subplots
ax = viz.boxplots(schw_sccoda, feature_name="Condition", y_scale='relative', add_dots=True, cmap='colorblind')
plt.ylim(-0.01, 0.9)
plt.xlabel('Cell cluster')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

for line in ax.get_lines():
    line.set_color('black')
# plt.show()